In [1]:
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, LSTM, GRU
from tensorflow.keras.initializers import Constant
from keras.layers import TextVectorization
import numpy as np
import pickle
import json

In [2]:
X = np.load('data/X.npy', allow_pickle=True)
y = np.load('data/y.npy')
embedding_matrix = np.load('data/embedding_matrix.npy')
num_tokens = np.load('data/num_tokens.npy').item()
embedding_dim = np.load('data/embedding_dim.npy').item()
max_tokens = np.load('data/max_tokens.npy').item()
max_len = np.load('data/max_len.npy').item()

In [3]:
vectorizer = TextVectorization(max_tokens=max_tokens,
                               standardize=None,
                               ngrams=(1, 2),
                               output_sequence_length=max_len)
vectorizer.adapt(X)

Metal device set to: Apple M1 Pro


2022-02-02 17:18:40.440483: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-02 17:18:40.440633: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-02-02 17:18:40.468995: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-02 17:18:40.505359: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [4]:
y = y.reshape((-1,1))

In [5]:
print(X.shape)
print(y.shape)

(7613,)
(7613, 1)


In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
np.save('data/x_train.npy', x_train)
np.save('data/y_train.npy', y_train)
np.save('data/x_test.npy', x_test)
np.save('data/y_test.npy', y_test)

In [8]:
sequences_input = Input(shape=(1), dtype='string')

vectorizer_sequence = vectorizer(sequences_input)

embedding_layer = Embedding(num_tokens,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            trainable=False)
embedded_sequences = embedding_layer(vectorizer_sequence)

x = Bidirectional(GRU(32, return_sequences=True))(embedded_sequences)
x = Bidirectional(GRU(18, return_sequences=True))(x)
x = Dense(10, activation="relu")(x)
x = Dropout(0.2)(x)

preds = Dense(units=1,
              kernel_initializer='uniform',
              activation='sigmoid')(x)
model = Model(sequences_input, preds)

model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 68)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 68, 100)           5000200   
                                                                 
 bidirectional (Bidirectiona  (None, 68, 64)           25728     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 68, 36)           9072      
 nal)                                                            
                                                             

In [9]:
n_samples = x_train.shape[0]
model_n = 6

history = model.fit(x_train,
                    y_train,
                    validation_split=0.2,
                    batch_size=int(n_samples*0.1),
                    epochs=100)

model.save(f'models/model{model_n}.tf')

with open(f'models/history{model_n}.json', 'w') as f:
    json.dump(history.history, f)

Epoch 1/100


2022-02-02 17:18:44.479680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:44.913719: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:44.928795: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:45.264787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:45.281995: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:45.593465: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:45.611791: I tensorflow/core/grappler/optimizers/cust

9/9 [==============================] - ETA: 0s - loss: 0.6915 - accuracy: 0.5602

2022-02-02 17:18:49.529960: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:49.679456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:49.688868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:49.867743: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 17:18:49.874107: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 9s 510ms/step - loss: 0.6915 - accuracy: 0.5602 - val_loss: 0.6882 - val_accuracy: 0.5762
Epoch 2/100
9/9 [==============================] - 1s 167ms/step - loss: 0.6876 - accuracy: 0.5621 - val_loss: 0.6829 - val_accuracy: 0.5763
Epoch 3/100
9/9 [==============================] - 1s 162ms/step - loss: 0.6838 - accuracy: 0.5623 - val_loss: 0.6774 - val_accuracy: 0.5787
Epoch 4/100
9/9 [==============================] - 1s 165ms/step - loss: 0.6805 - accuracy: 0.5658 - val_loss: 0.6724 - val_accuracy: 0.5868
Epoch 5/100
9/9 [==============================] - 2s 169ms/step - loss: 0.6761 - accuracy: 0.5779 - val_loss: 0.6664 - val_accuracy: 0.6023
Epoch 6/100
9/9 [==============================] - 1s 167ms/step - loss: 0.6704 - accuracy: 0.5934 - val_loss: 0.6602 - val_accuracy: 0.6121
Epoch 7/100
9/9 [==============================] - 1s 162ms/step - loss: 0.6653 - accuracy: 0.6010 - val_loss: 0.6513 - val_accuracy: 0.6167
Epoch 8/100
9/9 [========

2022-02-02 17:21:18.892244: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-02-02 17:21:24.455205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: models/model6.tf/assets


INFO:tensorflow:Assets written to: models/model6.tf/assets
